In [1]:
import pandas as pd
from collections import defaultdict
import numpy as np
from sortedcontainers import SortedList, SortedDict, SortedSet

In [2]:
item = pd.read_csv('Submissions/submission_item_item_500.txt', header=None)
knn = pd.read_csv('Submissions/submission_nearest_neighbor.txt', header=None)
pop = pd.read_csv('Submissions/submission_popularity.txt', header=None)

In [3]:
kaggle = pd.read_csv('kaggle_songs.txt', header=None, sep=' ')
kaggle_to_song = dict()
song_to_kaggle = dict()
for idx, row in kaggle.iterrows():
    kaggle_to_song[row[1]] = row[0]
    song_to_kaggle[row[0]] = row[1]

In [4]:
test_data = pd.read_csv('Submissions/submission_item_item_500.txt', header=None)

item_list = dict()
item_list = defaultdict(lambda:list(),item_list)

for i in range(len(test_data)):
    rec = list(test_data[0][i].split())
    new_rec = []
    for song in rec:
        new_rec.append(song)
    item_list[i] = set(new_rec)

In [5]:
test_data = pd.read_csv('Submissions/submission_nearest_neighbor.txt', header=None)

knn_list = dict()
knn_list = defaultdict(lambda:list(), knn_list)

for i in range(len(test_data)):
    rec = list(map(int, test_data[0][i].split()))
    new_rec = []
    for song in rec:
        new_rec.append(kaggle_to_song[song])
    knn_list[i] = set(new_rec)

In [6]:
test_data = pd.read_csv('Submissions/submission_popularity.txt', header=None)

pop_list = dict()
pop_list = defaultdict(lambda:list(), pop_list)

for i in range(len(test_data)):
    rec = list(map(int, test_data[0][i].split()))
    new_rec = []
    for song in rec:
        new_rec.append(kaggle_to_song[song])
    pop_list[i] = set(new_rec)

In [7]:
ensemble_rec = knn_list.copy()
for i in range(0, 10000):
    knn_songs = knn_list[i]
    item_songs = item_list[i]
    pop_songs = pop_list[i]
    
    intersection1 = knn_songs.intersection(item_songs).intersection(pop_songs)
    songs_to_recommend = list(intersection1)
    intersection2 = knn_songs.intersection(item_songs)
    for song in intersection2:
        if song not in songs_to_recommend:
            songs_to_recommend.append(song)
    intersection3 = knn_songs.intersection(pop_songs)
    for song in intersection3:
        if song not in songs_to_recommend:
            songs_to_recommend.append(song)
    intersection4 = item_songs.intersection(pop_songs)
    for song in intersection4:
        if song not in songs_to_recommend:
            songs_to_recommend.append(song)
    for song in knn_songs:
        if song not in songs_to_recommend:
            if len(songs_to_recommend) < 500: 
                songs_to_recommend.append(song)
    ensemble_rec[i] = songs_to_recommend    
    
#perform the different permutations of intersections and add them to the song recommendation for each user

In [13]:
test_user_list = dict()
test_user_list = defaultdict(lambda:list(), test_user_list)

for i in range(len(ensemble_rec.items())):
    new_rec = []
    for song in ensemble_rec[i]:
        test_user_list[i] = song

In [8]:
data = pd.read_csv('year1_valid_triplets_hidden.txt', header=None, sep='\t')

validation_user_list = dict()
validation_user_list = defaultdict(lambda:list(), validation_user_list)
last_user = 0
user_ct = -1

for i in range(len(data.iloc[:,0])):
    if data.iloc[i,0] != last_user:
        user_ct += 1
        validation_user_list[user_ct].append(data.iloc[i,1])
        last_user = data.iloc[i,0]
    else:
        validation_user_list[user_ct].append(data.iloc[i,1])

In [32]:
def compare(validation, test, quantity):
    accuracy_sum = 0
    for i in range(len(validation)):
        similar = len(np.intersect1d(validation[i],test[i][:quantity]))
        #accuracy_sum += similar / quantity
        accuracy_sum += similar / len(validation[i])
        
    return accuracy_sum / len(validation)
#performs recall or precision calculation

In [39]:
compare(validation_user_list, ensemble_rec, 500)

0.33179123477087896